In [2]:
import rasterio
import numpy as np

nir_image_path = "NIR_calibrada.tif"
red_image_path = "RED_calibrada.tif"
green_image_path = "GRE_calibrada.tif"
red_edge_image_path = "REG_calibrada.tif"

nir_image = rasterio.open(nir_image_path).read(1)
red_image = rasterio.open(red_image_path).read(1)
green_image = rasterio.open(green_image_path).read(1)
red_edge_image = rasterio.open(red_edge_image_path).read(1)


In [3]:
if nir_image.shape == red_image.shape == green_image.shape == red_edge_image.shape:
    print("Las imágenes tienen la misma forma.")
else:
    print("Las imágenes tienen formas diferentes. Verifica tus datos.")

if nir_image.dtype == red_image.dtype == green_image.dtype == red_edge_image.dtype:
    print("Las imágenes tienen el mismo tipo de datos.")
else:
    print("Las imágenes tienen tipos de datos diferentes. Verifica tus datos.")


Las imágenes tienen la misma forma.
Las imágenes tienen el mismo tipo de datos.


In [4]:
if rasterio.open(nir_image_path).transform == rasterio.open(red_image_path).transform == rasterio.open(green_image_path).transform == rasterio.open(red_edge_image_path).transform:
    print("Las imágenes tienen la corrección geométrica aplicada.")
else:
    print("Las imágenes no tienen la corrección geométrica aplicada. Verifica tus datos.")


Las imágenes tienen la corrección geométrica aplicada.


In [5]:
# Realiza una comprobación simple, por ejemplo, verifica si algún valor de píxel en las bandas es igual a cero o negativo.
if np.any(nir_image <= 0) or np.any(red_image <= 0) or np.any(green_image <= 0) or np.any(red_edge_image <= 0):
    print("Las imágenes no tienen la corrección de radiación aplicada. Verifica tus datos.")
else:
    print("Las imágenes tienen la corrección de radiación aplicada.")


Las imágenes no tienen la corrección de radiación aplicada. Verifica tus datos.


In [ ]:
import cv2
import numpy as np

# LECTURA DE LAS IMAGENES E HISTOGRAMA
I = cv2.imread('NIR_calibrada.tif', cv2.IMREAD_UNCHANGED)
cv2.imshow('ORTOMOSAICO BANDA INFRARROJO CERCANO', I[:, :, 1])
cv2.waitKey(0)
cv2.destroyAllWindows()

I2 = cv2.imread('RED.tif', cv2.IMREAD_UNCHANGED)
cv2.imshow('ORTOMOSAICO BANDA ROJA', I2[:, :, 1])
cv2.waitKey(0)
cv2.destroyAllWindows()

# CONVERSION A single
NIR = np.float32(I)
RED = np.float32(I2)

# NDVI
NDVI = (NIR - RED) / (NIR + RED)
cv2.imshow('ÍNDICE VEGETACION DIFERENCIA NORMALIZADO', NDVI[:, :, 1])
cv2.waitKey(0)
cv2.destroyAllWindows()

# SEGMENTACION OTSU INDICE DE VEGETACION DIFERENCIA NORMALIZADO
level1, segndvi = cv2.threshold(NDVI, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
EN1 = cv2.morphologyEx(segndvi[:, :, 1], cv2.MORPH_OPEN, np.ones((50, 50), np.uint8))
cv2.imshow('SEGMENTACION OTSU EN INDICE VEGETACION DIFERENCIA NORMALIZADO', EN1)
cv2.waitKey(0)
cv2.destroyAllWindows()

# VEGETACION SALUDABLE CON UMBRALIZACION BINARIA
umbrandvi = NDVI > 0.4
sano = cv2.morphologyEx(umbrandvi[:, :, 1], cv2.MORPH_OPEN, np.ones((50, 50), np.uint8))
# cv2.imshow('SEGMENTACION DEL CULTIVO SALUDABLE', sano)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# VEGETACION ESTRESADA VEGETACION SALUDABLE CON UMBRALIZACION BINARIA
umbrandvi1 = NDVI > 0.2
umbrandvi2 = NDVI < 0.4
umbraestresado = umbrandvi1 * umbrandvi2
nosano = cv2.morphologyEx(umbraestresado[:, :, 1], cv2.MORPH_OPEN, np.ones((50, 50), np.uint8))
# cv2.imshow('SEGMENTACION DEL CULTIVO ESTRESADO', nosano)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# SEGMENTACION OTSU INFRARROJO
level, segnir = cv2.threshold(I, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
EN3 = cv2.morphologyEx(segnir[:, :, 1], cv2.MORPH_OPEN, np.ones((50, 50), np.uint8))
# cv2.imshow('SEGMENTACION OTSU EN INFRARROJO CERCANO', EN3)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# LIMITACION DE INTERES DEL CULTIVO SALUDABLE
final1 = sano * EN3
# cv2.imshow('ZONA DE INTERES DEL CULTIVO SALUDABLE', final1[:, :, 1])
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# LIMITACION DE INTERES DEL CULTIVO ESTRESADO
final12 = nosano * EN3
cv2.imshow('ZONA DE INTERES DEL CULTIVO ESTRESADO', final12[:, :, 1])
cv2.waitKey(0)
cv2.destroyAllWindows()

# RESALTAR LA PARTE ESTRESADA EN COLORES
green = np.zeros(I.shape, dtype=np.uint8)
green[:, :, 1] = 255 * final12[:, :, 1]
alpha = final12[:, :, 1].astype(np.float32) / 255.0

result = I.copy()
result[final12 == False] = (result[final12 == False] * (1 - alpha)).astype(np.uint8)
result[final12 == True] += green[final12 == True]

cv2.imshow('Resaltado de la parte estres